## TIF Image Processing for CellPose

This reads in component images to create RGB images suitable for CellPose 2.0.  CellPose requires one color (G)
to represent cytoplasm and another (B) for nuclei. 

For each ROI sample image in the given Panel and component files dir, 
1. read in all the component_data.tif filenames
2. open each 10 channel tif into a numpy array with shape like (10, H, W)
3. re-scaling? Normalize across the batch, scale, and covert to 0-255 8 bit format. 
4. merge cytoplasm channels to one for green, take DAPI for blue, save as (H, W, 3) shaped numpy
5. save RGB tif files to RGB subdir.




### Todo yet
1. Get images and plots into single figures so they go on one PDF page
2. Only show the aggregated histograms per patient sample
3. Move many supporting functions to a .py import file to clean up 
4. Standardize folder names

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
# %matplotlib inline 

from skimage import io, color, filters, exposure
from skimage.transform import resize, rotate
import tifffile as tif
#import cv2

## Variables to set

These will vary by run so make sure these are correct:
* channels:  list of channel labels, in order of emissions range. 
* cyto_channels:  list of indices for channels that are good cytoplasm markers.  We include tumor and lymphcyte phenotypic markers vs. functional.   
* components_dir: input files copied from InForm export  
* output_dir: to contain RGB tifs, should be empty  



In [ ]:
# list of labels for channels
channels_p28 = ['CD3','pSTAT3 Y705', 'CD4', 'pSTAT5','pSTAT3 S727','pSTAT1', 'SOX10S100', 'CD8','DAPI','Autofluorescence']

channels_p68 = ['pSTAT6','SOX10S100','pSTAT4', 'CD4','CD3',
                'pSTAT1 S727', 'pSTAT2', 'CD8','DAPI','Autofluorescence']

cyto_channels_p28 = [0,2,6,7]  # CD3, CD4, SOX, CD8
cyto_channels_p68 = [1,3,4,7]  # SOX, CD4, CD3, CD8

channels=channels_p68
cyto_channels=cyto_channels_p68

print(channels)


In [ ]:
root_dir = '/Users/annmstrange/Documents/Projects/Tumor IF'
# sample_data_file = os.path.join(root_dir,'Analysis_Py/TumorIF_SampleData.xlsx')

#components_dir = "Panel2/020522 P28BatchAnalysis3x/components" # 3xblur
#components_dir = "Panel2/Export P28 full40x"  # vs 3x
components_dir = "Panel3/Export P68 full40x"
#components_dir = "Panel2/Export P28 21 full40x"

# output for RGB files
#output_dir = os.path.join(root_dir, 'Panel2/RGB_CellPose40x_23')
output_dir = os.path.join(root_dir, 'Panel3/RGB_CellPose40x_23')
#output_dir = os.path.join(root_dir, 'Panel2/RGB_CellPose40x_21')

In [ ]:
!pwd

In [ ]:
# For OneDrive file locations, uncomment the best line to use here

# !cd "cd ~/OneDrive\ -\ The\ University\ of\ Colorado\ Denver/Documents/Projects/Tumor\ IF/Panel2_Analysis "
# !cd "Panel2/020522 P28BatchAnalysis3x"

# We'll need 3x (low res files) from 020522\ P28BatchAnalysis3x/components, composites
#    binary masks, maybe qpTiff_as_tifs, binary_seg_maps

In [ ]:
# how os.path works...
# dir_name = os.path.basename("/Volumes/Glyph4TB/Projects/Tumor IF/Panel2/020522 P28BatchAnalysis3x")
# print(dir_name)
# print(os.path.dirname(dir_name)) 
# filename_suffix="tif"


In [ ]:
# code for recursively looking at files in subfolders
# builds lists of files we want to process
# simplest:
import os
import fnmatch
import re

def get_files_in_folder (src, pattern):
    '''
    Args: src is the full path where to look eg '/Volumes/Glyph4TB/Projects/Tumor IF/Panel2/013022 P28Images_full'
    pattern: string like '*_composite_image.tif' to use with fnmatch.filter
    Returns: list of full filenames
    '''
    # build list of filenames we want
    fname_list = []

    for dirpath, dir, files in os.walk(src):
        for filename in fnmatch.filter(files, pattern):
            fname = filename
            fullname = os.path.join(dirpath, filename)
            fname_list.append(fullname)
            #slideno_range = re.search(" [#][0-9][0-9]", fname).span()

            #print(fname[slideno_range[0]+2: slideno_range[1]])
            
    return fname_list       
 
 
fname_component = get_files_in_folder(os.path.join(root_dir, components_dir), 
                                      '*_component_data.tif') 

print('found {0} files matching the pattern'.format(len(fname_component)))
print(fname_component[0])



In [ ]:
# get a component from our data

component_file1 = fname_component[0]
component_file1

In [ ]:
# to get code working, forst get the 1st images from each list

img_component = io.imread(fname_component[0], plugin="tifffile")

In [ ]:
# Check the shapes

print('Note: we get 10 layers here, same as ImageJ BioFormats import (good)')
print('img_component shape {}'.format(img_component.shape))


In [ ]:

def get_max_intensities_by_channel (filename_list):
    '''
    Arguments: filename_list is list of component files
    Returns: array of length 10 with the max intensity in each channel
    '''

    tally_arr = np.zeros(10)
    #print(tally_arr.shape)
    for file in filename_list:
        img_arr = io.imread(file)
        max_values = np.max(np.max(img_arr, axis=2),axis=1)
        #print(max_values)
        #print(max_values.shape)
        tally_arr = np.max([tally_arr, max_values], axis = 0)
        #print(tally_arr)
        
    return tally_arr

print("checking {} images in list".format(len(fname_component)))
max_intensities = get_max_intensities_by_channel(fname_component)
print(max_intensities)

In [ ]:
# Rescale 16 bit image array

def rescale_img(img_array, max_intensities):
    '''
    Arguments: img_array is 10 channel array from Polaris
    max_intensities is array of 10 values for max channel values across the batch
    Returns: img_array with new intensities normalized within the channel.
    '''
    
    # rescaled to the 0-255 range, and change 16-bit to single bit unsigned int
    
    # This takes each array intensity basically div by array max to get a percent, then * 255
    img_rescaled = (255 * (img_array - img_array.min()) / (img_array.max() - img_array.min())).astype(np.uint8)
    
    # instead, we want to div by the channel batch max to get that percent.
    # e.g. if batch max is 100%, scale to total. 7.4 is x% of 83, so 7.4/83 will be the % of total possible intensity. 
    #     and the min() stuff is to rescale all the way to 0 in case of background "glow"
    
    # There's surely a way to do this in numpy using axis but it hurts my brain so I'll loop by channel
    for c in np.arange(10):
        channel_max = max_intensities[c]
        #print("For channel {} the max intensity is {}".format(c, channel_max))
        img_rescaled[c] = (255 * (img_array[c,:,:] - img_array[c,:,:].min()) / 
                           (channel_max - img_array[c,:,:].min())).astype(np.uint8)
        img_max = np.max(img_rescaled[c])
        print("For channel {} the max intensity in img is {} vs batch max {}".format(c, img_max, channel_max))
    return img_rescaled.astype(np.uint8)

    
    
reduced_component = rescale_img(img_component, max_intensities)
print('reduced_component shape {}'.format(reduced_component.shape))
print('Values like {}'.format(reduced_component[0,:5,:5]))
print('Max intensities now like {}'.format(reduced_component.max()))


In [ ]:
print(len(reduced_component))

In [ ]:
## Downsize and rescale to 8 bit (0-255)
def tiff_to_uint8 (tiff_array):
    '''
    Accept multilayer 16 bit tiff as numpy array.
    Shape should be 3D, similar to (10, 465, 620), aka not RGB (465,620,3)
    Returns: numpy array of same shape but with uint8 integer values (0-255) in RGB format
    
    Todo: check for valid input and output (values must be 0-255)
    '''
    
    # rescaled to the 0-255 range, and change 16-bit to single bit unsigned int
    img_rescaled = (255 * (tiff_array - tiff_array.min()) / (tiff_array.max() - tiff_array.min())).astype(np.uint8)
    
    return img_rescaled.astype(np.uint8)


reduced_component = tiff_to_uint8(img_component)
print('reduced_component shape {}'.format(reduced_component.shape))



In [ ]:
np.max(reduced_component[1,:,:])

In [ ]:
# select channels for nuclei and cytoplasm (for now, only one but you could take max intensities for mult)

# channel 8 (DAPI) to blue 
# channel idx 6 (SOX10S100) to green  (add CD3, CD4, and CD8 channels) pass [0,2,6,7]

def to_rgb (component_img, nuclear_idx, cyto_idx, include_red=True):
    '''
    Arguments: component_img as multichannel index with shape (channels, y, x)
        nuclear_idx is index of the channel containing best nuclear stain
        cyto_idx is index (or list of indices) of the channel containing best cytoplasm stain(s)
    Returns: RGB formatted numpy array w shape like (y, x, 3) which can be saved e.g. as a tif file
    '''
    
    # Scale each channel (0-1) then scale up to 0-255, HOWEVER, should be across whole batch.
    # soooo... we'll assume these are already scaled by channel and batch
    
    img_shape = component_img[nuclear_idx,:,:].shape
    rgb_np = np.zeros((img_shape[0], img_shape[1], 3))
    print(component_img[nuclear_idx,:,:].shape)
    
    # set nuclear channel to 2 which is blue in RGB
    rgb_np[:,:,2] = component_img[nuclear_idx,:,:] 
    
    if (len(cyto_idx) > 1 and type(cyto_idx == "list")):
        print("passed a list")
        merged_channels = np.max(component_img[cyto_idx,:,:], axis=0)
        print(merged_channels.shape)
        
        # red is everything left over
        red_idx = list(set([0,1,2,3,4,5,6,7]) - set(cyto_idx) - {nuclear_idx})
        print(red_idx)
        red_channels = np.max(component_img[red_idx,:,:], axis=0)

        #print("Channel maxs: {} {} {}".format(component_img[0,:,:].max(), component_img[2,:,:].max(), 
        #      component_img[6,:,:].max()))
        print("Merged max: {}".format(merged_channels.max()))
        # set cytoplasm channels to 1 which is green in RGB
        rgb_np[:,:,1] = merged_channels[:,:]      
    else:    
        rgb_np[:,:,1] = component_img[cyto_idx,:,:] 
        red_idx = list(set([0,1,2,3,4,5,6,7]) - {nuclear_idx})
        
    if (include_red):
        merged_channels = np.max(component_img[red_idx,:,:], axis=0)
        print(merged_channels.shape)
        rgb_np[:,:,0] = merged_channels[:,:]
    
    
    #print(rgb_np[:20, :20, 1])
    print(rgb_np.shape)
    return rgb_np.astype(np.uint8)


rgb_np1 = to_rgb(reduced_component, 8, cyto_channels, True) # 2
    


In [ ]:
# aside: how to subtract list elements from another list, use sets 
l1 = set([0,1,2,3,4,5,6,7])
l2 = [2, 7]
list(l1 - set(l2))


In [ ]:
rgb_np1 = to_rgb(reduced_component, 8, cyto_channels)
print(rgb_np1.shape)

print(np.min(rgb_np1), np.max(rgb_np1), type(rgb_np1[0,0,0]))

plt.imshow(rgb_np1)



In [ ]:
# process all files in the components list and save rgb files to new folder

for i, file1 in enumerate (fname_component):
    print(file1)
    img_component = io.imread(file1, plugin="tifffile")
    #reduced_component = tiff_to_uint8(img_component)
    reduced_component = rescale_img(img_component, max_intensities)
    
    rgb_arr = to_rgb(reduced_component, 8, cyto_channels)
    
    rgb_fn = os.path.basename(file1)
    rgb_file = os.path.join(output_dir, rgb_fn.replace('component_data', 'rgb'))
    print("Saving to {}".format(rgb_file))
    io.imsave(rgb_file, rgb_arr)

In [ ]:
len(fname_component)